In [ ]:
input_path = "input.mp4"
output_path = "output.mp4"

cap = cv2.VideoCapture(input_path)

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale = 4
out = cv2.VideoWriter(
	output_path,
	cv2.VideoWriter_fourcc(*"mp4v"),
	fps,
	(width*scale, height*scale)
)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

BATCH = 4
frame_buffer = []

with tqdm(total=total_frames) as pbar:
	while True:
		ret, frame = cap.read()
		if not ret:
			break

		frame_buffer.append(frame)

		if len(frame_buffer) == BATCH:
			output_batch = core_batch(model, frame_buffer)

			for out_frame in output_batch:
				out.write(out_frame)
				pbar.update(1)

			frame_buffer.clear()

	if len(frame_buffer) > 0:
		output_batch = core_batch(model, frame_buffer)
		for out_frame in output_batch:
			out.write(out_frame)
			pbar.update(1)

cap.release()
out.release()


In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2
!pip install realesrgan
!pip install opencv-python tqdm

import torch
import cv2
import numpy as np
from tqdm import tqdm
from realesrgan import RealESRGAN

# =========================
# モデルロード（CUDA）
# =========================
device = torch.device("cuda")
model = RealESRGAN(device, scale=4)
model.load_weights("RealESRGAN_x4plus.pth")

# =========================
# バッチ推論関数
# =========================
def core_batch(model, frames):
	# frames: list of HWC uint8 (BGR)

	# BGR → RGB
	frames = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames]

	arr = np.stack(frames, axis=0)              # [B,H,W,C]
	arr = arr.transpose(0,3,1,2)                # [B,C,H,W]
	arr = torch.from_numpy(arr).float().to(device)
	arr /= 255.0

	with torch.no_grad():
		output = model.model(arr)               # 直接forward

	output = torch.clamp(output, 0, 1)
	output = (output * 255).byte()
	output = output.permute(0,2,3,1).cpu().numpy()  # [B,H,W,C]

	# RGB → BGR
	output = [cv2.cvtColor(f, cv2.COLOR_RGB2BGR) for f in output]

	return output
